## System Architecture 12 - Deep Learning Software Programming

## (Part II) Model inference

Author: Qian Zhao and Noriyuki Kushiro, Department of Computer Science and Networks, Kyushu Institute of Technology, 2020.

---

After design and training of a nueual network, then we can use it in a real application. This jupyter notebook shows how to recognize a handwritten number use our LeNet-5 that trained with MNIST dataset.

## Prepare: Python packages imported below are required.
If you run the notebook on our prepared server, just push run button and enjoy.

If you wish to run this notebook on your own computer, you have to prepare python enviroment with these packages installed.

In [ ]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.layers import Dense, Flatten, Conv2D, AveragePooling2D
import matplotlib.pyplot as plt
from IPython.display import clear_output
from PIL import Image
import numpy as np
import cv2
from time import time
import os
import warnings; warnings.simplefilter('ignore')

In [ ]:
#os.environ['CUDA_VISIBLE_DEVICES'] = '-1' #comment out this line if you want to use GPU

if tf.test.gpu_device_name():
    print('GPU found')
    !nvidia-smi
else:
    print("No GPU found, use CPU")

In [ ]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

## Step 1. Create model and load our trained weights

We use the same LeNet-5 network structure.

In [ ]:
# LeNet-5 model as example
class LeNet(Sequential):
    def __init__(self, input_shape, nb_classes):
        super().__init__()

        self.add(Conv2D(6, kernel_size=(5, 5), strides=(1, 1), activation='relu', input_shape=input_shape))
        self.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))
        self.add(Conv2D(16, kernel_size=(5, 5), strides=(1, 1), activation='relu', padding='valid'))
        self.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))
        self.add(Flatten())
        self.add(Dense(120, activation='relu'))
        self.add(Dense(84, activation='relu'))
        self.add(Dense(nb_classes, activation='softmax'))

        self.compile(optimizer='adam',
                    loss=categorical_crossentropy,
                    metrics=['accuracy'])

First, create a model instance from LeNet class.

For this time, we load weights from file, so we do not have to retrain all the parameters for use.

Notice that, the neural network structure and its weights are a set. The weights is only meaningful for the same neural network from which it was exported.

In [ ]:
%%time

model = LeNet((28, 28, 1), 10)

#Load trained weights from file
model.load_weights("my_lenet_mnist_model.weight")

#Confirm the model
model.summary()

## Step 2. Draw a number with mouse as input data

Now, lets test our neural network! Run the cell below, and draw a number in the paint canvas.

In [ ]:
from IPython.display import HTML
from google.colab.output import eval_js
from base64 import b64decode

canvas_html = """
<div style="border: solid 2px #666; width: 143px; height: 144px; background: black;">
<canvas width=%d height=%d></canvas>
</div>
<button>Finish</button>
<script>
var canvas = document.querySelector('canvas')
var ctx = canvas.getContext('2d')
//ctx.lineWidth = %d
var button = document.querySelector('button')
var mouse = {x: 0, y: 0}
canvas.addEventListener('mousemove', function(e) {
        if (e.buttons == 1) {
            canvas.getContext("2d").fillStyle = "rgb(255,255,255)";
            canvas.getContext("2d").fillRect(e.offsetX, e.offsetY, 8, 8);
            x = Math.floor(e.offsetY * 0.2);
            y = Math.floor(e.offsetX * 0.2) + 1;
        }
})
canvas.onmousedown = ()=>{
  ctx.beginPath()
  ctx.moveTo(mouse.x, mouse.y)
  canvas.addEventListener('mousemove', onPaint)
}
canvas.onmouseup = ()=>{
  canvas.removeEventListener('mousemove', onPaint)
}
var onPaint = ()=>{
  ctx.lineTo(mouse.x, mouse.y)
  ctx.stroke()
}
var data = new Promise(resolve=>{
  button.onclick = ()=>{
    resolve(canvas.toDataURL('image/jpeg', 0.75))
  }
})
</script>
"""

def draw(filename='handwrite.jpg', w=140, h=140, line_width=1):
  display(HTML(canvas_html % (w, h, line_width)))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)

draw()

Now the image data you drawed will be saved in a plaintext file "handwrite.jpg". You can confirm the content of this file from jupyter file lists.

## Step 3. Read input data from file and convert to tensor format for model input

In [ ]:
%%time
#Load image handwrite.jpg
img = np.array(Image.open('handwrite.jpg'))
print(img.shape)

#Convert color from RGB to GRAY
img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
print(img.shape)

#Resize image to 28*28
img = cv2.resize(img, dsize=(28, 28))
print(img.shape)

#Reshape array hand_data to a tensor format that match our model's input
hand_input = np.array(img).reshape(1, 28,28,1)
print(hand_input.shape)

plt.imshow(img)

## Step 4. Perform model inference and show prediction results

Let's use our model recognize our handwritten number. It is very simple.

In [ ]:
%%time
#Do the model inference, predict the number from hand_input
result = model.predict(hand_input)

Now all the prediction values are in the "result" variable. There are 10 numbers in "result", each of which means a class for a number from 0 ~ 9, and its value means the possibility that the input image belong to this class.

In [ ]:
%%time
#Print predicted probability of 10 numbers (10 output classes of our model)
print(result[0])

#Show results in bar figure
fig = plt.figure(figsize=(6,4))
subplot = fig.add_subplot(1,1,1)
subplot.set_xticks(range(10))
subplot.set_xlim(-0.5,9.5)
subplot.bar(range(10), result[0], align='center')